In [1]:
#importing necessary libraries
import numpy as np
import matplotlib.pyplot as plt #only for visualization purposes, otherwise skip
import pandas as pd #only for saving simulation results in a dataframe, otherwise skip

In [2]:
#building a funciton with two arguments: "alarms" for a number of simultaneously triggered alarms, "runs" for a number of simulation runs
def func(alarms, runs):
    #list for collecting results for mean kinetic temperature exposure corrected by sampling rate, deivation duration, and cumulative mean kinetic temperature exposure for alarm processing manually, with k-NN, and Hk-NN
    results = [[], [], [], [], [], [], [], [], []]
    
    #declaring and initializing the variables that characterize the business context for simulation set-up
    #(to be initialized based on data from a business context before simulations can be run)
    
    #at first, the variables for a frequentist estimation of an alarm being triggered in a particular supply chain phase
    prec_al = prec_al #number of alarms within a studied time frame that were triggered in the phase "preconditioning"
    hand_al = hand_al #number of alarms within a studied time frame that were triggered in the phase "handling"
    flgt_al = flgt_al #number of alarms within a studied time frame that were triggered in the phase "flight"
    dlvd_al = dlvd_al #number of alarms within a studied time frame that were triggered in the phase "delivery"
    totl_al = prec_al + hand_al + flgt_al + dlvd_al #total number of alarms within a studied time frame
    
    #then, the variables for classification coverage by Hk-NN for alarm triggered in different supply chain phases
    prec_cov = prec_cov #number between 0 and 1.0 for prediction coverage by Hk-NN for preconditioning
    hand_cov = hand_cov #number between 0 and 1.0 for prediction coverage by Hk-NN for handling
    flgt_cov = flgt_cov #number between 0 and 1.0 for prediction coverage by Hk-NN for flight
    dlvd_cov = dlvd_cov #number between 0 and 1.0 for prediction coverage by Hk-NN for delivery
    
    #next, the variables for classification accuracy of k-NN and Hk-NN for alarms triggered in different supply chain phases
    acc_prec_k = acc_prec_k #accuracy of k-NN in a supply chain phase "preconditioning"
    acc_pred_h = acc_prec_h #accuracy of Hk-NN in a supply chain phase "preconditioning"
    acc_hand_k = acc_hand_k #accuracy of k-NN in a supply chain phase "handling"
    acc_hand_h = acc_hand_h #accuracy of Hk-NN in a supply chain phase "handling"
    acc_flgt_k = acc_flgt_k #accuracy of k-NN in a supply chain phase "flight"
    acc_flgt_h = acc_flgt_h #accuracy of Hk-NN in a supply chain phase "flight"
    acc_dlvd_k = acc_dlvd_k #accuracy of k-NN in a supply chain phase "delivery"
    acc_dlvd_h = acc_dlvd_h #accuracy of Hk-NN in a supply chain phase "delivery"
    
    sampling_rate = sampline_rate #frequency with which temperature measurements are collected and frequency with which alarms are triggered assuming no event data transmission delays (value between 0 and 1.0 with 1.0 staning for one hour)
    
    #probabilities for each supply chain phase of an alarm requiring a specific corrective measure (estimated in a frequentist way based on historical data and having values between 0.0 and 1.0 in a format .xxx)
    #first, for supply chain phase 'preconditionig'
    prec_nan = prec_nan #probability of an event that a triggered alarm is not commented by alarm personnel
    prec_act = prec_act #probability of an event that a triggered alarm requires 'action' as corrective measure
    prec_inv = prec_inv #probability of an event that a triggered alarm requires 'investigation' as corrective measure
    prec_mon = prec_mon #probability of an event that a triggered alarm requires 'monitoring' as corrective measure
    prec_nac = prec_nac #probability of an event that a triggered alarm requires 'no action' as corrective measure
    
    #then, for supply chain phase 'handling' (variables are named according to the principle for supply chain phase 'preconditioning')
    hand_nan = hand_nan
    hand_act = hand_act
    hand_inv = hand_inv
    hand_mon = hand_mon
    hand_nac = hand_nac
    
    #then, for supply chain phase 'flight' (same naming convention)
    flgt_nan = flgt_nan
    flgt_act = flgt_nan
    flgt_inv = flgt_inv
    flgt_mon = flgt_mon
    flgt_nac = flgt_nac
    
    #finally, for supply chain phase 'delivery' (also, same naming convention)
    dlvd_nan = dlvd_nan
    dlvd_act = dlvd_act
    dlvd_inv = dlvd_inv
    dlvd_mon = dlvd_mon
    dlvd_nac = dlvd_nac
    
    #loop for a scpesified number of simulation runs
    for i in range(runs):
        #counters for mean kinetic temperature exposure (corrected by a sampling rate) for alarms triggered in one simulation run
        t_kin_knn = 0 #for k-NN
        t_kin_hyb = 0 #for Hk-NN
        t_kin_not = 0 #for manual processing
        
        #counters for cumulative time with registered temperature deviation
        time_not = 0 #for manual processing
        time_knn = 0 #for k-NN
        time_hyb = 0 #for Hk-NN
        
        success = 0 #variable for simulating a stochastic probability of a correct classification of alarm
        extra_time = 0 #variable taking values for extra alarm processing time, e.g. in cases of incorrect classifications
        acc_knn_list = [] #list with the accuracy of alarm classification by k-NN per triggered alarm
        acc_hyb_list = [] #list with the accuracy of alarm classification by Hk-NN per triggered alarm
        cov_hyb_list = [] #list with prediction coverage for triggered and classified alarms by Hk-NN
        true_cm_list = [] #list containing true correcive measures for triggered alarms (later appended based on
        #the frequencies of historically true corrective measures for alarms in different supply chain phases)
        inv_tim_list = [] #list containing initial investigation times for triggered alarms
        
        #lists with cumulative mean kinetic temperature exposure for each alarm separately
        temp_not = [0]*alarms #manual processing
        temp_knn = [0]*alarms #processing based on k-NN prediction
        temp_hyb = [0]*alarms #processing based on Hk-NN prediction
        
        temp = 0 #current temperature deviation (variable taking values for deviations during initial examination and during corrective measures)

        active_knn = [1]*alarms #list storing boolean values for an alarm state, i.e. '1' for active, '0' for processed (for alarm classified by k-NN)
        active_hyb = [1]*alarms #list storing boolean values for an alarm state, i.e. '1' for active, '0' for processed (for alarm classified by Hk-NN)
        covered_hyb = [] #list storing boolean values for prediction coverage success for Hk-NN, i.e. '1' for successful classificatoin, '0' for relegation to manual processing
        inv_time_hyb = 0 #initial alarm examination time for Hk-NN
        
        #alarm processing times depending on the way of processing (k-NN, Hk-NN, manually) and expected corrective measure
        init_inv_time = min(np.random.gamma(shape, scale), higher_bound) #random variable for initial alarm examination time irrespective of the way of processing with a higher bound; should be approximated from collected data, e.g. min(np.random.gamma(2, 0.15), 2.5)
        extra_time_act_inv = min(np.random.gamma(shape, scale), higher_bound) #random variable for additional alarm examination time if it eventually required 'action' or 'investigation' as a corrective measure with a higher bound; should also be approximated from collected data, e.g. min(np.random.gamma(2, 0.1), 1.5)
        extra_time_others = min(np.random.gamma(shape, scale), higher_bound) #random variable for additional alarm examinatino time if it eventually requires 'monitoring' or no corrective measure with a higher bound; should also be approximated from collected data, e.g. min(np.random.gamma(2, 0.05), 1.0)

        #estimated temperature deviations depending on the corrective measure (lower and higher bounds below are represented in absolute setpoint deviations; e.g. for a temperature regime +2-+8C, temperature of +9C amounts to 0.333 (temp_over_higher_threshold/(higher_threshold - setpoint)))
        temp_init = np.random.uniform(low, high) #random variable for initial temperature deviation after alarm triggering with lower and higher bounds irrespective of corrective measure required afterwards; should be approximated from collected data, e.g. np.random.uniform(0.1, 2.0)
        temp_act_inv = np.random.uniform(low, high) #random variable for temperature deviation after taking corrective measure 'action' or 'investigation' with lower and higher bounds that is always lower than the initial deviation temperature prior to alarm triggering; should be approximated from collected data, e.g. np.random.uniform(0.1, 0.3)
        temp_others = np.random.uniform(low, high) #radom variable for temperature deviation after taking corrective measure 'monitoring' or no measure with lower and higher bounds that is always lower than the initial deviation temperature prior to alarm triggering; should be approximated from collected data, e.g. np.random.uniform(0, 0.1)

        #generating alarms for different supply chain phases based on their historical frequencies
        phase_digit = list(np.random.randint(totl_al, size = alarms)) #random digit identifying the grouping of alarms
        for element in phase_digit: 
            if element < prec_al: #for supply chain phase 'preconditioning'
                acc_knn_list.append(acc_pred_k)
                acc_hyb_list.append(acc_pred_h)
                cov_hyb_list.append(prec_cov)
                success = np.random.uniform() #probability of a non-problematic alarm, i.e. that can be classified by Hk-NN and not relegated to manual processing
                if success <= prec_cov:
                    covered_hyb.append(1)
                else:
                    covered_hyb.append(0)
                inv_tim_list.append(init_inv_time)
                al = np.random.randint(1000)
                if al < prec_nan*1000:
                    true_cm_list.append('nan') #true corrective measure is not given
                elif al >= prec_nan*1000 and al < (prec_nan + prec_inv)*1000:
                    true_cm_list.append('inv') #true corrective measure is 'investigation'
                elif al >= (prec_nan + prec_inv)*1000 and al < (prec_nan + prec_inv + prec_nac)*1000:
                    true_cm_list.append('no_act') #true corrective measure is 'no action'
                elif al >= (prec_nan + prec_inv + prec_nac)*1000 and al < (prec_nan + prec_inv + prec_nac + prec_act)*1000:
                    true_cm_list.append('act') #true corrective measure is 'action'
                else:
                    true_cm_list.append('mon') #true corrective measure is 'monitoring'

            elif element >= prec_al and element < (prec_al + hand_al): #supply chain phase 'handling'
                acc_knn_list.append(acc_hand_k)
                acc_hyb_list.append(acc_hand_h)
                cov_hyb_list.append(hand_cov)
                success = np.random.uniform()
                if success <= hand_cov:
                    covered_hyb.append(1)
                else:
                    covered_hyb.append(0)
                inv_tim_list.append(init_inv_time)
                al = np.random.randint(1000)
                if al < hand_nan*1000:
                    true_cm_list.append('nan') #true corrective measure is not given
                elif al >= hand_nan*1000 and al < (hand_nan + hand_inv)*1000:
                    true_cm_list.append('inv') #true corrective measure is 'investigation'
                elif al >= (hand_nan + hand_inv)*1000 and al < (hand_nan + hand_inv + hand_nac)*1000:
                    true_cm_list.append('no_act') #true corrective measure is 'no action'
                elif al >= (hand_nan + hand_inv + hand_nac)*1000 and al < (hand_nan + hand_inv + hand_nac + hand_act)*1000:
                    true_cm_list.append('act') #true corrective measure is 'action'
                else:
                    true_cm_list.append('mon') #true corrective measure is 'monitoring'

            elif element >= (prec_al + hand_al) and element < (prec_al + hand_al + flgt_al): #supply chain phase 'flight'
                acc_knn_list.append(acc_flgt_k)
                acc_hyb_list.append(acc_flgt_h)
                cov_hyb_list.append(flgt_cov)
                success = np.random.uniform()
                if success <= flgt_cov:
                    covered_hyb.append(1)
                else:
                    covered_hyb.append(0)
                inv_tim_list.append(init_inv_time)
                al = np.random.randint(1000)
                if al < flgt_nan*1000:
                    true_cm_list.append('nan') #true corrective measure is not given
                elif al >= flgt_nan*1000 and al < (flgt_nan + flgt_inv)*1000:
                    true_cm_list.append('inv') #true corrective measure is 'investigation'
                elif al >= (flgt_nan + flgt_inv)*1000 and al < (flgt_nan + flgt_inv + flgt_nac)*1000:
                    true_cm_list.append('no_act') #true corrective measure is 'no action'
                elif al >= (flgt_nan + flgt_inv + flgt_nac)*1000 and al < (flgt_nan + flgt_inv + flgt_nac + flgt_act)*1000:
                    true_cm_list.append('act') #true corrective measure is 'action'
                else:
                    true_cm_list.append('mon') #true corrective measure is 'monitoring'

            else: #supply chain phase 'delivery'
                acc_knn_list.append(acc_dlvd_k)
                acc_hyb_list.append(acc_dlvd_h)
                cov_hyb_list.append(dlvd_cov)
                success = np.random.uniform()
                if success <= dlvd_cov:
                    covered_hyb.append(1)
                else:
                    covered_hyb.append(0)
                inv_tim_list.append(init_inv_time)
                al = np.random.randint(1000)
                if al < dlvd_nan*1000:
                    true_cm_list.append('nan') #true corrective measure is not given
                elif al >= dlvd_nan*1000 and al < (dlvd_nan + dlvd_inv)*1000:
                    true_cm_list.append('inv') #true corrective measure is 'investigation'
                elif al >= (dlvd_nan + dlvd_inv)*1000 and al < (dlvd_nan + dlvd_inv + dlvd_nac)*1000:
                    true_cm_list.append('no_act') #true corrective measure is 'no action'
                elif al >= (dlvd_nan + dlvd_inv + dlvd_nac)*1000 and al < (dlvd_nan + dlvd_inv + dlvd_nac + dlvd_act)*1000:
                    true_cm_list.append('act') #true corrective measure is 'action'
                else:
                    true_cm_list.append('mon') #true corrective measure is 'monitoring'
                    
#We now have lists for true corrective measures and associated accuracy and prediction coverage values
#We will go along each of three scenarios separately

        #generating true corrective measure lists for each type of processing (manually, with k-NN, and with Hk-NN) that is a duplication based on corrective measures estimated with the help of their historical frequencies
        not_list = [x for x in true_cm_list]
        knn_list = [x for x in true_cm_list]
        hyb_list = [x for x in true_cm_list]
        
#first, considering the scenario without any algorithm (manual processing of temperature alarms)
        for j in range(len(not_list)):
            temp = temp_init
            t_kin_not += sum(inv_tim_list[:(j+1)])*temp
            time_not += sum(inv_tim_list[:(j+1)])
            temp_not[j] = (j+1)*temp
            if not_list[j] == 'act' or not_list[j] == 'inv':
                extra_time = extra_time_act_inv
                time_not += extra_time
                temp = temp_act_inv
                t_kin_not += extra_time*temp
                temp_not[j] = temp_not[j] + temp 
            else:
                extra_time = extra_time_others
                time_not += extra_time
                temp = temp_others
                t_kin_not += extra_time*temp
                temp_not[j] = temp_not[j] + temp
                
#second, considering the scenario with alarm processing based on predictions by k-NN
        for j in range(len(knn_list)):
            if knn_list[j] == 'act' or knn_list[j] == 'inv':
                extra_time = extra_time_act_inv
                time_knn += extra_time
                temp = temp_act_inv
                t_kin_knn += extra_time*temp
                temp_knn[j] = temp_knn[j] + temp
            else:
                extra_time = extra_time_others
                time_knn += extra_time
                temp = temp_others
                t_kin_knn += extra_time*temp
                temp_knn[j] = temp_knn[j] + temp
            success = np.random.uniform()
            if success <= acc_knn_list[j]:
                active_knn[j] = 0
                
#working with still active alarms for k-NN (for the cases of wrong predictions suggesting no corrective measures)
        temp = temp_init
        t_kin_knn += active_knn.count(1)*temp*sampling_rate
        time_knn += active_knn.count(1)*sampling_rate
        for m in range(len(active_knn)):
            if active_knn[m] == 1:
                temp_knn[m] = temp_knn[m] + temp
        k = 0
        while active_knn != [0]*alarms:
            success = np.random.uniform()
            if success <= acc_knn_list[k]:
                active_knn[k] = 0
            if k < (len(active_knn) - 1):
                k += 1
            else:
                k = 0
            temp = temp_init
            t_kin_knn += active_knn.count(1)*temp*sampling_rate
            time_knn += active_knn.count(1)*sampling_rate
            for n in range(len(active_knn)):
                if active_knn[n] == 1:
                    temp_knn[n] = temp_knn[n] + temp
                    
#lastly, considering the scenario with alarm processing based on predictions by Hk-NN
        inv_time_hyb = init_inv_time
        time_hyb += inv_time_hyb*covered_hyb.count(0)
        temp = temp_init
        t_kin_hyb += covered_hyb.count(0)*temp*inv_time_hyb
        for o in range(len(covered_hyb)):
            if covered_hyb[o] == 0:
                temp_hyb[o] = temp_hyb[o] + temp
        for l in range(len(covered_hyb)):
            if covered_hyb[l] == 0:
                active_hyb[l] = 0
        for j in range(len(hyb_list)):
            if active_hyb[j] != 0:
                if hyb_list[j] == 'act' or hyb_list[j] == 'inv':
                    extra_time = extra_time_act_inv
                    time_hyb += extra_time
                    temp = temp_act_inv
                    t_kin_hyb += extra_time*temp
                    temp_hyb[j] = temp_hyb[j] + temp
                else:
                    extra_time = extra_time_others
                    time_hyb += extra_time
                    temp = temp_others
                    t_kin_hyb += extra_time*temp
                    temp_hyb[j] = temp_hyb[j] + temp
                success = np.random.uniform()
                if success <= acc_hyb_list[j]:
                    active_hyb[j] = 0
                    
#working with still active alarms for Hk-NN (for the cases of wrong predictions suggesting no corrective measures)
        temp = temp_init
        t_kin_hyb += active_hyb.count(1)*temp*sampling_rate
        time_hyb += active_hyb.count(1)*sampling_rate
        for p in range(len(active_hyb)):
            if active_hyb[p] == 1:
                temp_hyb[p] = temp_hyb[p] + temp
        k = 0
        while active_hyb != [0]*alarms:
            success = np.random.uniform()
            if success <= acc_hyb_list[k]:
                active_hyb[k] = 0
            if k < (len(active_hyb) - 1):
                k += 1
            else:
                k = 0
            temp = temp_init
            t_kin_hyb += active_hyb.count(1)*temp*sampling_rate
            time_hyb += active_hyb.count(1)*sampling_rate
            for u in range(len(active_hyb)):
                if active_hyb[u] == 1:
                    temp_hyb[u] = temp_hyb[u] + temp
                    
#appending the output of our function
        results[0].append(t_kin_not)
        results[1].append(t_kin_knn)
        results[2].append(t_kin_hyb)
        results[3].append(time_not)
        results[4].append(time_knn)
        results[5].append(time_hyb)
        results[6].append(temp_not)
        results[7].append(temp_knn)
        results[8].append(temp_hyb)
    
    return results